In [ ]:
pip install split-folders

In [1]:
import zipfile
# import splitfolders
import os
import tensorflow as tf

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Extract the archive
local_zip = '/content/drive/MyDrive/dataset_capstone/224x224_5class.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [26]:
# delete class
!rm -rf '/content/data/test/Oblong'
!rm -rf '/content/data/test/Oval'
!rm -rf '/content/data/train/Oblong'
!rm -rf '/content/data/train/Oval'

In [ ]:
# base_dir = 'data/dataset_capstone/'
# splitfolders.ratio(base_dir, output = 'data/dataset_capstone/dataset', seed = 1337, ratio=(0.8, 0.2), group_prefix=None)
# train_dir = 'data/dataset_capstone/dataset/train'
# val_dir = 'data/dataset_capstone/dataset/val'

In [27]:
!unzip /content/drive/MyDrive/dataset_capstone/224x224_5class.zip -d /content

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: /content/train/Round/Round__0_8618.jpg  
  inflating: /content/train/Round/Round__0_1725.jpg  
  inflating: /content/train/Round/Round__0_6280.jpg  
  inflating: /content/train/Round/Round__0_3623.jpg  
  inflating: /content/train/Round/Round__0_8605.jpg  
  inflating: /content/train/Round/Round__0_7705.jpg  
  inflating: /content/train/Round/Round__0_6414.jpg  
  inflating: /content/train/Round/Round__0_4730.jpg  
  inflating: /content/train/Round/Round__0_908.jpg  
  inflating: /content/train/Round/Round__0_6747.jpg  
  inflating: /content/train/Round/Round__0_8311.jpg  
  inflating: /content/train/Round/Round__0_6514.jpg  
  inflating: /content/train/Round/Round__0_2832.jpg  
  inflating: /content/train/Round/Round__0_6356.jpg  
  inflating: /content/train/Round/Round__0_3312.jpg  
  inflating: /content/train/Round/Round__0_4054.jpg  
  inflating: /content/train/Round/Round__0_3450.jpg  
  inflating: /content/tr

In [29]:
!mkdir /content/temp
!mkdir /content/temp2

In [32]:
!mv /content/train/{Oblong,Oval} /content/temp2
!mv /content/test/{Oblong,Oval} /content/temp2

mv: cannot stat '/content/train/Oblong': No such file or directory
mv: cannot stat '/content/train/Oval': No such file or directory


In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=10,
      # width_shift_range=0.3,
      # height_shift_range=0.3,
      # shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='reflect')

validation_datagen = ImageDataGenerator(
                    rescale=1./255)

In [34]:
train_dir = '/content/train'
val_dir = '/content/test'

In [52]:
!ls -1 /content/train/Heart | wc -l
!ls -1 /content/test/Heart | wc -l
!ls -1 /content/train/Round | wc -l
!ls -1 /content/test/Round | wc -l
!ls -1 /content/train/Square | wc -l
!ls -1 /content/test/Square | wc -l


2124
588
2113
1029
2145
581


In [35]:
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(224,224),
	class_mode='sparse',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	val_dir,
	target_size=(224,224),
	class_mode='sparse',
  batch_size=32
)

Found 6382 images belonging to 3 classes.
Found 2198 images belonging to 3 classes.


In [44]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [37]:
local_zip = '/content/drive/MyDrive/dataset_capstone/model.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/dataset_capstone/model')
zip_ref.close()

In [54]:
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_capstone/saved_model_Dec_16_2023_nv2')

In [55]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [56]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])

In [57]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [58]:
# Train the model
history = model.fit(train_generator, epochs=10, validation_data = validation_generator, verbose = 1)

Epoch 1/10
200/200 [==============================] - 93s 454ms/step - loss: 0.1295 - accuracy: 0.9517 - val_loss: 0.8671 - val_accuracy: 0.7520
Epoch 2/10
200/200 [==============================] - 93s 466ms/step - loss: 0.1239 - accuracy: 0.9544 - val_loss: 0.7782 - val_accuracy: 0.7561
Epoch 3/10
200/200 [==============================] - 92s 458ms/step - loss: 0.1129 - accuracy: 0.9604 - val_loss: 0.8841 - val_accuracy: 0.7511
Epoch 4/10
200/200 [==============================] - 97s 481ms/step - loss: 0.1128 - accuracy: 0.9588 - val_loss: 0.8510 - val_accuracy: 0.7589
Epoch 5/10
200/200 [==============================] - 95s 474ms/step - loss: 0.1037 - accuracy: 0.9643 - val_loss: 0.8437 - val_accuracy: 0.7566
Epoch 6/10
200/200 [==============================] - 97s 486ms/step - loss: 0.1025 - accuracy: 0.9636 - val_loss: 0.8285 - val_accuracy: 0.7589
Epoch 7/10
200/200 [==============================] - 94s 472ms/step - loss: 0.0999 - accuracy: 0.9652 - val_loss: 0.8510 - val_ac

In [59]:
tf.keras.saving.save_model(model, '/content/drive/MyDrive/dataset_capstone/saved_model_Dec_16_2023_nv3')

In [ ]:
pip install tensorflowjs

In [ ]:
import tensorflowjs

In [ ]:
import time
saved_model_path = "./{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

In [ ]:
saved_model_path = "./{}.h5".format(int(time.time()))

history.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import os
import random

# Load your saved model
# model_path = 'path/to/your/saved_model'
loaded_model =  model #tf.keras.models.load_model(model_path)

# Specify the directory containing your test images
test_directory = '/content/train/Heart'
# test_directory = '/content/train/Round'
# test_directory = '/content/train/Square'

# Get a list of image files in the test directory
image_files = [f for f in os.listdir(test_directory) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Randomly select 10 samples
random_10_samples = random.sample(image_files, 10)

# Class labels corresponding to Heart, Round, Square
class_labels = ['Heart', 'Round', 'Square']

# Iterate over the 10 randomly selected image files and make predictions
for img_file in random_10_samples:
    img_path = os.path.join(test_directory, img_file)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Make a prediction
    predictions = loaded_model.predict(img_array)

    # Get the predicted class index (0, 1, or 2)
    predicted_class_index = np.argmax(predictions)

    # Get the corresponding class label
    predicted_class_label = class_labels[predicted_class_index]

    # Print the prediction result
    print(f"\nPredicted face shape for {img_file}: {predicted_class_label}")


1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_9993.jpg: Round
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_3462.jpg: Round
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_6314.jpg: Heart
1/1 [==============================] - 0s 23ms/step

Predicted face shape for Heart__0_2852.jpg: Round
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_7532.jpg: Heart
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_1292.jpg: Round
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_6812.jpg: Heart
1/1 [==============================] - 0s 18ms/step

Predicted face shape for Heart__0_5819.jpg: Round
1/1 [==============================] - 0s 19ms/step

Predicted face shape for Heart__0_8928.jpg: Heart
1/1 [==============================] - 0s 18ms/step

Predicted face shape

In [ ]:
import os

def count_files_in_directory(directory_path):
    # Get the list of files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # Count the number of files
    num_files = len(files)

    return num_files

# Specify the directory path
directory_path = '/content/test/Heart'

# Call the function to count files
num_files = count_files_in_directory(directory_path)

# Print the result
print(f"Number of files in the directory: {num_files}")


Number of files in the directory: 588


In [ ]:
!zip /content/model.zip -r /content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5

  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/variables.data-00000-of-00001 (deflated 19%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/variables/variables.index (deflated 65%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/assets/ (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/saved_model.pb (deflated 87%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/fingerprint.pb (stored 0%)
  adding: content/drive/MyDrive/style-mate/saved_model_Dec_16_2023_v5/keras_metadata.pb (deflated 92%)


In [ ]:
!cp /content/model.zip /content/drive/MyDrive/style-mate/